<a href="https://colab.research.google.com/github/yashugupta786/Agents-notes/blob/main/fara_updated.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Check GPU
!nvidia-smi

import sys, platform, torch

print("Python version:", sys.version)
print("Platform:", platform.platform())
print("PyTorch CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("CUDA device name:", torch.cuda.get_device_name(0))


Fri Dec  5 13:39:42 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   32C    P0             43W /  400W |       0MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [2]:
%%bash
set -euxo pipefail

apt-get update

# Basic tools
apt-get install -y git wget curl

# Common headless browser deps (Playwright / Firefox)
apt-get install -y \
  libnss3 \
  libatk1.0-0 \
  libatk-bridge2.0-0 \
  libxkbcommon0 \
  libgtk-3-0 \
  libx11-xcb1 \
  libxcb1 \
  libxcomposite1 \
  libxdamage1 \
  libxfixes3 \
  libxrandr2 \
  libgbm1 \
  libpango-1.0-0 \
  libcairo2 \
  fonts-liberation


Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://cli.github.com/packages stable InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Hit:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Reading package lists...
Reading package lists...
Building dependency tree...
Reading state information...
curl is already the newest version (7.81.0-1ubuntu1.21).
git is already the newest version (1:2.34.1-1ubuntu1.15).
wget is alr

+ apt-get update
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
+ apt-get install -y git wget curl
+ apt-get install -y libnss3 libatk1.0-0 libatk-bridge2.0-0 libxkbcommon0 libgtk-3-0 libx11-xcb1 libxcb1 libxcomposite1 libxdamage1 libxfixes3 libxrandr2 libgbm1 libpango-1.0-0 libcairo2 fonts-liberation


In [3]:
%%bash
set -euxo pipefail

cd /content

if [ ! -d "fara" ]; then
  git clone https://github.com/microsoft/fara.git
else
  echo "Repo already cloned, pulling latest..."
  cd fara
  git pull
fi

cd /content/fara
ls


Repo already cloned, pulling latest...
Already up to date.
autogen
CODE_OF_CONDUCT.md
endpoint_configs
figures
LICENSE
pyproject.toml
README.md
scripts
SECURITY.md
src
TRANSPARENCY_NOTE.md
webeval


+ cd /content
+ '[' '!' -d fara ']'
+ echo 'Repo already cloned, pulling latest...'
+ cd fara
+ git pull
+ cd /content/fara
+ ls


In [4]:
%cd /content/fara

# Upgrade pip first
!python -m pip install --upgrade pip

# Install fara in editable mode with vLLM support
!pip install -e .[vllm]

# Install Playwright browsers (Firefox) for Python
!python -m playwright install firefox

# Install system deps for Playwright (extra safety in containers)
!python -m playwright install-deps firefox


/content/fara
Obtaining file:///content/fara
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Installing backend dependencies ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for fara (pyproject.toml) ... done
  Created wheel for fara: filename=fara-0.1.0-py3-none-any.whl size=10316 sha256=691023c3195c4915893e630d7ef3caa0a4212b77ebeaf842b92d7c5425df9d8d
  Stored in directory: /tmp/pip-ephem-wheel-cache-e1q_x1np/wheels/a8/25/c1/9e5075f254df0d1cb0691a8042fc3856680abd79c7b405b7ac
Successfully built fara
  Attempting uninstall: fara
    Found existing installation: fara 0.1.0
    Uninstalling fara-0.1.0:
      Successfully uninstalled fara-0.1.0
Installing dependencies...
Hit:1 https://cli.github.com/packages stable InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 https://cloud.r-project.org/b

In [5]:
import fara
from playwright.async_api import async_playwright
import vllm

print("Fara version imported OK.")
print("vLLM version:", vllm.__version__)


Fara version imported OK.
vLLM version: 0.12.0


In [6]:
%%bash
set -euxo pipefail

cd /content/fara

mkdir -p logs

# To avoid fork issues in notebook environments
export VLLM_WORKER_MULTIPROC_METHOD=spawn

# Start vLLM in background with logs
nohup vllm serve "microsoft/Fara-7B" \
  --port 5000 \
  --host 0.0.0.0 \
  --dtype auto \
  --max-model-len 8192 \
  --gpu-memory-utilization 0.9 \
  > logs/vllm.log 2>&1 &

# Give it some time to start (it will also download weights from HF)
sleep 30

echo "=== vLLM processes ==="
ps aux | grep vllm | head


=== vLLM processes ===
root       13033 71.1  1.8 13107964 1605832 ?    Sl   13:41   0:21 /usr/bin/python3 /usr/local/bin/vllm serve microsoft/Fara-7B --port 5000 --host 0.0.0.0 --dtype auto --max-model-len 8192 --gpu-memory-utilization 0.9
root       13170  127  1.2 6451304 1071296 ?     Rl   13:41   0:08 /usr/bin/python3 -m vllm.model_executor.models.registry
root       13217  0.0  0.0   6484  2576 ?        S    13:41   0:00 grep vllm


+ cd /content/fara
+ mkdir -p logs
+ export VLLM_WORKER_MULTIPROC_METHOD=spawn
+ VLLM_WORKER_MULTIPROC_METHOD=spawn
+ sleep 30
+ nohup vllm serve microsoft/Fara-7B --port 5000 --host 0.0.0.0 --dtype auto --max-model-len 8192 --gpu-memory-utilization 0.9
+ echo '=== vLLM processes ==='
+ ps aux
+ grep vllm
+ head


In [12]:
%%bash
set -euxo pipefail
cd /content/fara

echo "=== Last 50 lines of vLLM log ==="
tail -n 50 logs/vllm.log || echo "No log yet (maybe still starting)"

echo
echo "=== Query /v1/models to verify vLLM is up ==="
curl -s http://localhost:5000/v1/models | python -m json.tool || echo "vLLM not ready yet"


=== Last 50 lines of vLLM log ===
Capturing CUDA graphs (decode, FULL): 100%|██████████| 35/35 [00:01<00:00, 17.72it/s]
(EngineCore_DP0 pid=13344) INFO 12-05 13:45:22 [gpu_model_runner.py:4466] Graph capturing finished in 6 secs, took 0.56 GiB
(EngineCore_DP0 pid=13344) INFO 12-05 13:45:22 [core.py:254] init engine (profile, create kv cache, warmup model) took 59.91 seconds
(APIServer pid=13033) INFO 12-05 13:45:24 [api_server.py:1520] Supported tasks: ['generate']
(APIServer pid=13033) WARNING 12-05 13:45:25 [model.py:1576] Default sampling parameters have been overridden by the model's Hugging Face generation config recommended from the model creator. If this is not intended, please relaunch vLLM instance with `--generation-config vllm`.
(APIServer pid=13033) INFO 12-05 13:45:25 [serving_responses.py:194] Using default chat sampling params from model: {'repetition_penalty': 1.05, 'temperature': 1e-06}
(APIServer pid=13033) INFO 12-05 13:45:25 [serving_chat.py:133] Using default chat 

+ cd /content/fara
+ echo '=== Last 50 lines of vLLM log ==='
+ tail -n 50 logs/vllm.log
+ echo
+ echo '=== Query /v1/models to verify vLLM is up ==='
+ curl -s http://localhost:5000/v1/models
+ python -m json.tool


In [13]:
%%bash
set -euxo pipefail
cd /content/fara

echo "=== Fara CLI help ==="
fara-cli --help || echo "fara-cli not found"

echo
echo "=== Python module entrypoint sanity ==="
python -m fara.run_fara --help || echo "python -m fara.run_fara failed"


=== Fara CLI help ===
usage: fara-cli [-h] [--task TASK] [--start_page START_PAGE] [--headful]
                [--downloads_folder DOWNLOADS_FOLDER] [--save_screenshots]
                [--max_rounds MAX_ROUNDS] [--browserbase]
                [--endpoint_config ENDPOINT_CONFIG] [--api_key API_KEY]
                [--base_url BASE_URL] [--model MODEL]

Run FARA agent interactively

options:
  -h, --help            show this help message and exit
  --task TASK           Initial task for the FARA agent (optional)
  --start_page START_PAGE
                        The starting page
  --headful             Run the browser in headful mode (show GUI, default is
                        headless)
  --downloads_folder DOWNLOADS_FOLDER
                        Folder to save screenshots and downloads
  --save_screenshots    Whether to save screenshots during the agent's
                        operation
  --max_rounds MAX_ROUNDS
                        Maximum number of rounds for the agent to run

+ cd /content/fara
+ echo '=== Fara CLI help ==='
+ fara-cli --help
+ echo
+ echo '=== Python module entrypoint sanity ==='
+ python -m fara.run_fara --help


In [16]:
%%bash
cd /content/fara

mkdir -p logs runs/smoke

TASK="What is the current population of India? Use the web, navigate as needed, and give a concise numeric answer with a short explanation."
LOG_FILE="logs/fara_smoke.log"
DOWNLOADS_DIR="runs/smoke"

echo "=== Running Fara smoke test ==="
echo "Task: $TASK"
echo "Log file: $LOG_FILE"
echo "Downloads folder: $DOWNLOADS_DIR"
echo

if command -v fara-cli >/dev/null 2>&1; then
  echo "[INFO] Using fara-cli"
  fara-cli \
    --task "$TASK" \
    --max_rounds 20 \
    --save_screenshots \
    --downloads_folder "$DOWNLOADS_DIR" \
    2>&1 | tee "$LOG_FILE"
else
  echo "[WARN] fara-cli not found, falling back to python -m fara.run_fara"
  python -m fara.run_fara \
    --task "$TASK" \
    --max_rounds 20 \
    --save_screenshots \
    --downloads_folder "$DOWNLOADS_DIR" \
    2>&1 | tee "$LOG_FILE"
fi

STATUS=$?
echo
echo "=== Fara exit status: $STATUS ==="

echo
echo "=== Last 80 lines of $LOG_FILE ==="
tail -n 80 "$LOG_FILE" || echo "Log file not found"


=== Running Fara smoke test ===
Task: What is the current population of India? Use the web, navigate as needed, and give a concise numeric answer with a short explanation.
Log file: logs/fara_smoke.log
Downloads folder: runs/smoke

[INFO] Using fara-cli
Initializing Browser...
Browser Running... Starting Fara Agent...
##########################################
Task: What is the current population of India? Use the web, navigate as needed, and give a concise numeric answer with a short explanation.
##########################################
Running Fara...


Thought #1: Search for the current population of India to get a numeric value
Action #1: executing tool 'type' with arguments {"action": "type", "coordinate": [701, 205], "text": "current population of India", "press_enter": true, "delete_existing_text": false}
Observation#1: I typed 'current population of India'.

Thought #2: Open the Worldometer result to view the latest estimate for India’s population
Action #2: executing tool 'l

In [17]:
#----- Complex output

In [18]:
%%bash
cd /content/fara

mkdir -p logs runs/amazon_headphones

TASK="Act as a shopping assistant.
Go to https://www.amazon.in.
Search for 'noise cancelling headphones under 5000 rupees'.
Open at least 2–3 product pages in new tabs or by navigating back and forth.
Compare their prices, ratings, and any key pros/cons you can see on the page.
Do NOT attempt to log in or purchase anything.
At the end, give:
1) A short table with Product Name, Approx Price, Rating (if visible).
2) Your recommendation for ONE product, with a 3–4 line justification."

LOG_FILE="logs/fara_amazon_headphones.log"
DOWNLOADS_DIR="runs/amazon_headphones"

echo "=== Fara Shopping Test 1: Amazon Headphones ==="
echo "Task: $TASK"
echo "Log file: $LOG_FILE"
echo "Downloads folder: $DOWNLOADS_DIR"
echo

fara-cli \
  --task "$TASK" \
  --max_rounds 45 \
  --save_screenshots \
  --downloads_folder "$DOWNLOADS_DIR" \
  2>&1 | tee "$LOG_FILE"

STATUS=$?
echo
echo "=== Fara exit status (Amazon Headphones): $STATUS ==="
echo
echo "=== Last 100 lines of $LOG_FILE ==="
tail -n 100 "$LOG_FILE" || echo "Log file not found"


=== Fara Shopping Test 1: Amazon Headphones ===
Task: Act as a shopping assistant. 
Go to https://www.amazon.in. 
Search for 'noise cancelling headphones under 5000 rupees'. 
Open at least 2–3 product pages in new tabs or by navigating back and forth. 
Compare their prices, ratings, and any key pros/cons you can see on the page. 
Do NOT attempt to log in or purchase anything. 
At the end, give:
1) A short table with Product Name, Approx Price, Rating (if visible).
2) Your recommendation for ONE product, with a 3–4 line justification.
Log file: logs/fara_amazon_headphones.log
Downloads folder: runs/amazon_headphones

Initializing Browser...
Browser Running... Starting Fara Agent...
##########################################
Task: Act as a shopping assistant. 
Go to https://www.amazon.in. 
Search for 'noise cancelling headphones under 5000 rupees'. 
Open at least 2–3 product pages in new tabs or by navigating back and forth. 
Compare their prices, ratings, and any key pros/cons you can s

In [19]:
%%bash
cd /content/fara

mkdir -p logs runs/amazon_add_to_cart

TASK="End-to-end add-to-cart test (no login).
You are a browser-control agent.
Steps:
1) Go to https://www.amazon.in.
2) In the search bar, search for: 'wireless mouse for laptop'.
3) From the search results, pick ONE reasonably priced product that clearly looks like a wireless mouse (avoid sponsored banners if possible).
4) Open its product detail page.
5) Scroll enough to confirm it is a wireless mouse (see title/description).
6) Click the 'Add to Cart' button.
7) IMPORTANT:
   - Do NOT attempt to sign in.
   - If a login/sign-in page appears at any point, STOP and do NOT enter any credentials.
   - Do NOT proceed to address or payment.
8) At the end, in your Final Answer, explain:
   - Product name and approximate price you tried to add.
   - What happened after clicking 'Add to Cart' (e.g., mini-cart on the right, redirect to sign-in, etc.).
   - Whether, based on the screen you see, it LOOKS like the item was added to cart or blocked by login."

LOG_FILE="logs/fara_amazon_add_to_cart.log"
DOWNLOADS_DIR="runs/amazon_add_to_cart"

echo "=== Fara Add-to-Cart Test (Amazon Wireless Mouse) ==="
echo "Task: $TASK"
echo "Log file: $LOG_FILE"
echo "Downloads folder: $DOWNLOADS_DIR"
echo

fara-cli \
  --task "$TASK" \
  --max_rounds 60 \
  --save_screenshots \
  --downloads_folder "$DOWNLOADS_DIR" \
  2>&1 | tee "$LOG_FILE"

STATUS=$?
echo
echo "=== Fara exit status (Add-to-Cart): $STATUS ==="
echo
echo "=== Last 100 lines of $LOG_FILE ==="
tail -n 100 "$LOG_FILE" || echo "Log file not found"


=== Fara Add-to-Cart Test (Amazon Wireless Mouse) ===
Task: End-to-end add-to-cart test (no login).
You are a browser-control agent.
Steps:
1) Go to https://www.amazon.in.
2) In the search bar, search for: 'wireless mouse for laptop'.
3) From the search results, pick ONE reasonably priced product that clearly looks like a wireless mouse (avoid sponsored banners if possible).
4) Open its product detail page.
5) Scroll enough to confirm it is a wireless mouse (see title/description).
6) Click the 'Add to Cart' button.
7) IMPORTANT:
   - Do NOT attempt to sign in.
   - If a login/sign-in page appears at any point, STOP and do NOT enter any credentials.
   - Do NOT proceed to address or payment.
8) At the end, in your Final Answer, explain:
   - Product name and approximate price you tried to add.
   - What happened after clicking 'Add to Cart' (e.g., mini-cart on the right, redirect to sign-in, etc.).
   - Whether, based on the screen you see, it LOOKS like the item was added to cart or b